# 미니 프로젝트 : 반자동 문서 스캐너

### 12에서 4개의 좌표를 일일이 가져오는 게 아니라 클릭을 4번하면 자동으로 가져오게 하자

### 마우스 이벤트 

In [7]:
import cv2

def mouse_handler(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN: # 마우스 좌클릭
        print('왼쪽 버튼 Down')
        print(x, y)
    elif event == cv2.EVENT_LBUTTONUP: # 마우스 좌클릭후 뗌
        print('왼쪽 버튼 Up')
        print(x, y)
    elif event == cv2.EVENT_LBUTTONDBLCLK: # 마우스 좌클릭 더블클릭 : 더블 클릭의 발생 시점은 좌클다운 - 업 - 더블클릭 - 업 임
        print('Double Click')
    # elif event == cv2.EVENT_MOUSEMOVE: # 마우스 이동
    #     print('마우스 이동')
    elif event == cv2.EVENT_RBUTTONDOWN:
        print('마우스 우클릭')

img = cv2.imread('Image/poker.jpg')

cv2.namedWindow('img') # 창(윈도우)만 만듦 : img라는 이름. 여기에 마우스 이벤트를 처리하기 위한 핸들러 적용 가능

cv2.setMouseCallback('img', mouse_handler)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

왼쪽 버튼 Down
880 361
왼쪽 버튼 Up
880 361
Double Click
왼쪽 버튼 Up
880 361


## 프로젝트 코드

In [3]:
import cv2
import numpy as np

point_lst = []
src_img = cv2.imread('Image/poker.jpg')

COLOR = (255, 0, 255)
THICKNESS = 3 
drawing = False # 선을 그릴 지 여부 (클릭 이후에 선이 그어져야 한다)

# 마우스 클릭 - 좌표를 리스트에 저장 - 4개가 저장되면 원근 변환을 통해 펼쳐진 이미지를 보여줄 것
def mouse_handler(event, x, y, flags, param):
    global drawing
    dst_img = src_img.copy() # src_img를 쓰면 마우스 커서가 지나간 곳마다 선이 남음

    if event == cv2.EVENT_LBUTTONDOWN: # 마우스 좌클릭
        drawing = True # 선을 그리기 시작
        point_lst.append((x, y))
    
    if drawing:
        prev_point = None # 직선의 시작점
        # 클릭한 지점을 이미지에 표시
        for point in point_lst:
            cv2.circle(dst_img, point, 10, COLOR, cv2.FILLED)
            if prev_point:
                cv2.line(dst_img, prev_point, point, COLOR, THICKNESS, cv2.LINE_AA)
            prev_point = point

        # 현재 마우스 이동 위치까지 선을 그어주는 작업
        next_point = (x, y) # 현재 마우스 핸들러에서 받아온 좌표
        if len(point_lst) == 4:
            show_result(point_lst) # 결과 출력 함수
            next_point = point_lst[0] # 1번째 클릭한 점으로 돌려줌
        cv2.line(dst_img, prev_point, next_point, COLOR, THICKNESS, cv2.LINE_AA) # 마지막 점 -> 처음 점 이어줌
        
    cv2.imshow('img', dst_img)

def show_result(point_lst):
    width, height = 530, 710
    src = np.float32(point_lst)
    dst = np.array([[0,0], [width,0], [width, height], [0,height]], dtype = np.float32)
    matrix = cv2.getPerspectiveTransform(src, dst)
    result = cv2.warpPerspective(src_img, matrix, (width, height))

    cv2.imshow('result', result)


cv2.namedWindow('img') # 창(윈도우)만 만듦 : img라는 이름. 여기에 마우스 이벤트를 처리하기 위한 핸들러 적용 가능

cv2.setMouseCallback('img', mouse_handler) # 마우스의 반응을 봄 + 반응에 따른 작동하게끔 함

cv2.imshow('img', src_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


### 여기서 좀 더 나아가면 완전 자동 스캐너도 만들 수가 있음
- 뒤에서 나오는 내용을 활용하면 되는데 이 강의는 초보자 대상이라서요 머쓱 ㅎㅎ